In [1]:
import win32com.client as win32
import pandas as pd
import numpy as np

In [2]:
def APIGetData(streamid, fromDate, toDate, displayID=0, dataInterval=0, newData=True):

    # The following 2 lines are the bare minimum to get data from the NRGSTREAM API

    obj = win32.Dispatch('NRGSTREAM.Trader.API.Extractor')# Replace Trader with Xtractor to use Xtractor API
    curs = obj.APIGetData(streamid,fromDate, # Call the NRGSTREAM API, returns an array
                          toDate,displayID,
                          dataInterval,newData)
    #print(curs)
    # Additional sample code to massage the data into a Pandas series
    
    x = np.asarray(curs[0][4:]) # Put first row which is dates starting at column 4 to exclude headers into an array
    y = np.asarray(curs[1][4:]) # Put second row which is data starting at column 4 to exclude headers into another array
    
    ser = pd.Series(index=x, data=y)# Create a pandas series with the dates as the index
    ser.replace(r'^\s*$', np.nan, regex=True, inplace=True) #this line is added by me
    ser = ser.astype('float64') # set the data type to float64
    ser.fillna(method='pad',inplace= True) # This line is added by me

    ser.index = pd.to_datetime(ser.index)   # convert the index to date time
    
    return ser 
# data = APIGetData('155938:1',-200,-197)      # Stream ID with a trailing :1 means master stream id

In [3]:
from datetime import datetime
def days_between(d1, d2):
    d1 = datetime.strptime(d1, "%Y-%m-%d")
    d2 = datetime.strptime(d2, "%Y-%m-%d")
    return abs((d2 - d1).days)

#### Select desired windfarms

In [5]:
windfarms = pd.read_excel('NRGstreamData/WindFarms.xlsx').sort_values(by='Asset ID').reset_index(drop = True)
desired = list(windfarms['Asset ID'])[:]
print(desired)

# desired = ['Load'] #  
desired = ['Price']

['AKE1', 'ARD1', 'BSR1', 'BTR1', 'BUL1', 'BUL2', 'CR1', 'CRE3', 'CRR1', 'GWW1', 'HAL1', 'IEW1', 'IEW2', 'KHW1', 'NEP1', 'OWF1', 'SCR2', 'SCR3', 'SCR4', 'TAB1']


### Download  data

In [15]:
# https://www.nrgstream.com/installation.aspx
fromDate = '2013-01-01'
toDate = '2013-12-31'
dataInterval=60 #minutes to download data (it can not be changed, if changed, does not have any impact !)
n_days = days_between(fromDate,toDate)

resampling = False
resampling_period = '5T'

save_folder = fromDate + ' to ' + toDate

for item in desired:
    if item=='Load':
        streamID = '225:1'  
    elif item=='Price':
        streamID = '2:1'    
        
    elif item != 'Load':
        ind = windfarms.index[windfarms['Asset ID'] == item].tolist()
        streamID = (windfarms.loc[ind]['StreamID'])
        streamID = str(int(streamID)) + ':1'     
        print(item, 'with stream ID = ' + streamID)
    
        

    
    try:
        data = APIGetData(streamID,fromDate,toDate, dataInterval=5, newData=False)# APIGetData('155938:1',-200,-197) 
        print('downloaded records = ', data.shape[0], '\n')

        # Truncutae data (resample data)
        if resampling: data = data.resample(resampling_period).mean()

        # assert(data.shape[0] == int((n_days+1) * 24 * (60/dataInterval)))
    except:
#         data = pd.DataFrame()
        print('Error :) \n')
        
    data.to_csv(item + '_' + fromDate + '_' + toDate + '_' + str(dataInterval) + '.csv')

downloaded records =  8760 



### codes shows why I had to add to lines of codes to APIGetData 

In [ ]:
s = pd.Series(['0', '2','', 4])
s.replace(r'^\s*$', np.nan, regex=True, inplace=True)
s.astype('float64')
s.fillna(method='pad',inplace= True)

s

In [ ]:
s.astype('float64')
s.fillna(method='pad',inplace= True)
s